## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [ ]:
# requests 라이브러리 설치여부 확인
%pip show requests

In [ ]:
# beautifulsoup4 라이브러리 설치여부 확인
%pip show beautifulsoup4

In [ ]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [ ]:
# requests, bs4 버전 확인하기
print(f'requests 버전 {requests.__version__}')
print(f'beautifulsoup4 버전 {bs4.__version__}')

### 1. 뉴스 제목 추출하기

In [ ]:
# IT/과학 뉴스
req_param = {"sid": 105}
#
url = "https://news.naver.com/section/{sid}".format(**req_param)
print(url)

# url2 = 'https://news.naver.com/section/105'

# 요청 헤더 설정 : 브라우저 정보 (사람처럼 보이게 하려고 설정하는 것임)
req_header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}

# requests 의 get() 함수 호출하기
res = requests.get(url, headers=req_header)
print(res)
print(type(res))
print(res.status_code, res.ok)

# 응답(response)이 OK 이면
if res.ok:
    # 응답 (response)에서 text 추출 - source 보기
    html = res.text
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html, "html.parser")
    # print(soup)
    # CSS 선택자
    # <div class='sa_text'><a href=''></a></div>
    print(len(soup.select("div.sa_text a[href*='mnews/article']")))
    a_tag_list = soup.select("div.sa_text a[href*='mnews/article']")
    print(
        type(a_tag_list)
    )  # [<a href="http://">뉴스제목</a>] ResultSet객체 , <a></a> : Tag 객체
    for a_tag in a_tag_list:
        # print(type(a_tag))
        title = a_tag.text.strip()
        # href 속성의 값을 가져오기
        link = a_tag["href"]
        print(link, title)
else:
    # 응답(response)이 Error 이면 status code 출력
    print(f"에러 코드 = {res.status_code}")

### 1.1 뉴스제목 추출하는 함수 선언하기

In [ ]:
import requests
from bs4 import BeautifulSoup

section_dict = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}

def print_news(sid):  # print_new(103)
    section = section_dict[sid]
    url = f"https://news.naver.com/section/{sid}"
    print(f"=====> {url} {section} 뉴스 <=====")

    # 요청 헤더 설정 : 브라우저 정보 (사람처럼 보이게 하려고 설정하는 것임)
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }

    # requests 의 get() 함수 호출하기
    res = requests.get(url, headers=req_header)
    print(res)
    print(type(res))
    print(res.status_code, res.ok)

    # 응답(response)이 OK 이면
    if res.ok:
        # 응답 (response)에서 text 추출 - source 보기
        html = res.text
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(html, "html.parser")
        
        print(len(soup.select("div.sa_text a[href*='mnews/article']")))
        a_tag_list = soup.select("div.sa_text a[href*='mnews/article']")    
        print(type(a_tag_list))  # [<a href="http://">뉴스제목</a>] ResultSet객체 , <a></a> : Tag 객체
        
        for a_tag in a_tag_list:
            # print(type(a_tag))
            title = a_tag.text.strip()
            # href 속성의 값을 가져오기
            link = a_tag["href"]
            print(link, title)
    else:
        # 응답(response)이 Error 이면 status code 출력
        print(f"에러 코드 = {res.status_code}")

# 함수호출
print_news(101)

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [ ]:
import requests
import os

req_header = {
    'referer': 'https://comic.naver.com/webtoon/detail?titleId=798173&no=5&amp;weekday=thu'
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기 
    res = requests.get(img_url, headers=req_header)
    if res.ok:    
        # binary 응답 데이터 가져오기 content 프로퍼티를 사용
        img_data =res.content
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        # binday data를 file에 write하기
        with open(file_name,'wb') as file:
            print(f'Writing to {file_name}({len(img_data):,} bytes) ')
            file.write(img_data)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=35&week=tue'
req_header = {
    'referer': url
}
res = requests.get(url)
if res.ok:
    # jpg 파일의 절대경로 url를 찾기
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("img[src*='IMAG01']"))) #19
    print(len(soup.select("img[src$='.jpg']"))) #71
    print(len(soup.select("img[src^='https://image-comic']"))) #71
    
    img_tags = soup.select("img[src*='IMAG01']")
    # img src 를 저장할 list 선언
    img_url_list = list() # ['aa.jpg','aa1.jpg']
    for img_tag in img_tags:
        #print(type(img_tag), img_tag.name, img_tag)
        img_src = img_tag['src']
        img_url_list.append(img_src)
    print(len(img_url_list), img_url_list[:2]) 
    
    #img 디렉토리가 없으면 생성하기   
    imgdir_name = 'img'
    if not os.path.isdir(imgdir_name):
        os.mkdir(imgdir_name)
        
    #os.path.join(dir,file) 함수 사용하여 디렉토리명과 파일명 합치기
    for img_url in img_url_list:
        res = requests.get(img_url, headers=req_header)
        if res.ok:        
            # binary data 가져오기
            img_data = res.content
            # img/xxx.jpg 디렉토리명과 파일명을 join
            dir_name = os.path.join(imgdir_name, os.path.basename(img_url))
            # binday data를 file에 write하기
            with open(dir_name,'wb') as file:
                print(f'writing to {dir_name}({len(img_data):,} bytes)')
                file.write(img_data)
        else:
            print(f'Error Code = {res.status_code}')      
            
else:
    print(f'Error Code = {res.status_code}')

#### 리팩토링 된 코드

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

# 기본 설정
url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=35&week=tue'
req_header = {'referer': url}
imgdir_name = 'img'

# 이미지 저장 폴더가 없으면 생성
os.makedirs(imgdir_name, exist_ok=True)

# 웹 페이지 요청 및 확인
res = requests.get(url)
if not res.ok:
    print(f'Error Code = {res.status_code}')
    exit()

# 이미지 URL 추출
soup = BeautifulSoup(res.text, 'html.parser')
img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

# 이미지 다운로드
for img_url in img_url_list:
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        img_data = res.content
        file_path = os.path.join(imgdir_name, os.path.basename(img_url))
        with open(file_path, 'wb') as file:
            print(f'Writing to {file_path} ({len(img_data):,} bytes)')
            file.write(img_data)
    else:
        print(f'Error Code = {res.status_code} for {img_url}')


### 3. 파일 업로드 하기
* [httpbin](https://httpbin.org/) 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    'img1': open('f1.jpg','rb'),
    'img2': open('f2.jpg','rb'),
}
print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.
res = requests.post(url,files=upload_files)
print(res.status_code)
#print(res.json())
print(res.json()['files']['img1'])

### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청
import urllib.request

client_id = "" # 개발자센터에서 발급받은 Client ID 값
client_secret = "" # 개발자센터에서 발급받은 Client Secret 값
code = "0"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code
# Request 객체생성성
request = urllib.request.Request(url)

#요청 헤더 설정
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

# 요청 보내기
response = urllib.request.urlopen(request)

# 응답 코드 가져오기기
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [ ]:
# 캡차 이미지 요청
import urllib.request

client_id = "" # 개발자센터에서 발급받은 Client ID 값
client_secret = "" # 개발자센터에서 발급받은 Client Secret 값
key = "" # 캡차 Key 값

url = "https://openapi.naver.com/v1/captcha/ncaptcha.bin?key=" + key
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    print("캡차 이미지 저장")
    response_body = response.read()
    with open('captcha.jpg', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

In [ ]:
# 사용자 입력값 검증 요청
import urllib.request

client_id = "" # 개발자센터에서 발급받은 Client ID 값
client_secret = "" # 개발자센터에서 발급받은 Client Secret 값
key = "" # 캡차 Key 값

code = "1"
value = "X19L3K"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code + "&key=" + key + "&value=" + value
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
%pip show python-dotenv

In [ ]:
# 캡차 키 발급 요청
import os
from dotenv import load_dotenv
import requests

# .env 파일에서 환경 변수 로드
load_dotenv()

client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID 값
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET 값
code = "0"

url = f"https://openapi.naver.com/v1/captcha/nkey?code={code}"

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 요청 보내기
response = requests.get(url, headers=headers)

# 응답 처리
if response.status_code == 200:
    print(response.text)
else:
    print(f"Error Code: {response.status_code}")



In [ ]:
# 캡차 이미지 요청
import os
from dotenv import load_dotenv
import requests

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET
key = os.getenv("NAVER_CAPTCHA_KEY")  # .env 파일의 NAVER_CAPTCHA_KEY

url = f"https://openapi.naver.com/v1/captcha/ncaptcha.bin?key={key}"

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 요청 보내기
response = requests.get(url, headers=headers)

# 응답 처리
if response.status_code == 200:
    print("캡차 이미지 저장")
    with open('captcha.jpg', 'wb') as f:
        f.write(response.content)
else:
    print(f"Error Code: {response.status_code}")

In [ ]:
# 사용자 입력값 검증 요청
import os
from dotenv import load_dotenv
import requests

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET
key = os.getenv("NAVER_CAPTCHA_KEY")  # .env 파일의 NAVER_CAPTCHA_KEY

# 사용자 입력값
code = "1"
value = "93S5VBK"  # 실제 사용시 사용자 입력값으로 대체 필요

# URL 구성
# query string key1=value1&key2=value2
url = f"https://openapi.naver.com/v1/captcha/nkey?code={code}&key={key}&value={value}"

# 요청 헤더
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 요청 보내기
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # 4xx, 5xx 오류 발생시 예외 발생
    
    # 응답 처리
    print(response.text)
    
except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")
    print(f"Status Code: {response.status_code if 'response' in locals() else 'N/A'}")

### 5. 블로그 검색하기

In [ ]:
import requests
import pprint
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# query string 문자열을 dict 선언
payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json' #?query=파이썬&display=100&sort=sim

# requests get(url, params, headers) 요청 
res = requests.get(url, params=payload, headers=headers)
# json() 함수로 응답 결과 가져오기
items_data = res.json()['items']
#print(type(res.json()))
#print(len(items_data), type(items_data), type(items_data[0]))

#[{},{},{}] 
# [] - 100 개의 데이터, {title:'',link:''} - 1개의 블로그 글 데이터

# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'
items_list = list()
item_list = []
for item in items_data[:2]:
#     print(item)
    item_list.append(item['title'])
    item_list.append(item['bloggername'])
    item_list.append(item['description'])
    item_list.append(item['bloggerlink'])
    item_list.append(item['link'])

    items_list.append(item_list)
    item_list = []

print(items_list) #[[],[]]

# 'data/nhnblog.txt' 파일 생성하기
with open('data/nhnblog.txt','w',encoding="utf-8")as file:
    for items in items_list:
        for item in items:
            item = item + '\n'
            file.write(item)
        file.write('-'*150+'\n')
